In [35]:
import pandas as pd 

from great_expectations.data_context import FileDataContext


## Data Loading

In [36]:
df = pd.read_csv('retail_store_inventory.csv')

### Data exploration

The following is prelimenary explorations for checks of eronious/problematic data.. Mising data/incorrect field types, and also to plan next steps

In [37]:
df.columns

Index(['Date', 'Store ID', 'Product ID', 'Category', 'Region',
       'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast',
       'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion',
       'Competitor Pricing', 'Seasonality'],
      dtype='object')

### Cleaning column header

In [38]:
df.columns = df.columns.str.replace(' ', '_').str.replace('/', 'or')

In [39]:
df.columns

Index(['Date', 'Store_ID', 'Product_ID', 'Category', 'Region',
       'Inventory_Level', 'Units_Sold', 'Units_Ordered', 'Demand_Forecast',
       'Price', 'Discount', 'Weather_Condition', 'HolidayorPromotion',
       'Competitor_Pricing', 'Seasonality'],
      dtype='object')

Printing out the first few data to see data structure

In [40]:
df.head()

,Date,Store_ID,Product_ID,Category,Region,Inventory_Level,Units_Sold,Units_Ordered,Demand_Forecast,Price,Discount,Weather_Condition,HolidayorPromotion,Competitor_Pricing,Seasonality
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer
3,2022-01-01,S001,P0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn
4,2022-01-01,S001,P0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer


In [41]:
df.tail()

,Date,Store_ID,Product_ID,Category,Region,Inventory_Level,Units_Sold,Units_Ordered,Demand_Forecast,Price,Discount,Weather_Condition,HolidayorPromotion,Competitor_Pricing,Seasonality
73095,2024-01-01,S005,P0016,Furniture,East,96,8,127,18.46,73.73,20,Snowy,0,72.45,Winter
73096,2024-01-01,S005,P0017,Toys,North,313,51,101,48.43,82.57,10,Cloudy,0,83.78,Autumn
73097,2024-01-01,S005,P0018,Clothing,West,278,36,151,39.65,11.11,10,Rainy,0,10.91,Winter
73098,2024-01-01,S005,P0019,Toys,East,374,264,21,270.52,53.14,20,Rainy,0,55.80,Spring
73099,2024-01-01,S005,P0020,Groceries,East,117,6,165,2.33,78.39,20,Rainy,1,79.52,Spring


from the following we can see that the date is sequential

In [42]:
df.nunique()

Date                    731
Store_ID                  5
Product_ID               20
Category                  5
Region                    4
Inventory_Level         451
Units_Sold              498
Units_Ordered           181
Demand_Forecast       31608
Price                  8999
Discount                  5
Weather_Condition         4
HolidayorPromotion        2
Competitor_Pricing     9751
Seasonality               4
dtype: int64

from the following we can see that the likely columns that are categorical are for Seasonality, HolidayorPromotion, Weather_Condition, Discount, Region, Category,  product ID and store ID

## EDA

## PM Data Engineering(Data Exploration)

### Check for missing Values

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                73100 non-null  object 
 1   Store_ID            73100 non-null  object 
 2   Product_ID          73100 non-null  object 
 3   Category            73100 non-null  object 
 4   Region              73100 non-null  object 
 5   Inventory_Level     73100 non-null  int64  
 6   Units_Sold          73100 non-null  int64  
 7   Units_Ordered       73100 non-null  int64  
 8   Demand_Forecast     73100 non-null  float64
 9   Price               73100 non-null  float64
 10  Discount            73100 non-null  int64  
 11  Weather_Condition   73100 non-null  object 
 12  HolidayorPromotion  73100 non-null  int64  
 13  Competitor_Pricing  73100 non-null  float64
 14  Seasonality         73100 non-null  object 
dtypes: float64(3), int64(5), object(7)
memory usage: 8.4+

from the following there seems to be no null instance so we can move on.

In [52]:
df.nunique()

Date                    731
Store_ID                  5
Product_ID               20
Category                  5
Region                    4
Inventory_Level         451
Units_Sold              498
Units_Ordered           181
Demand_Forecast       31608
Price                  8999
Discount                  5
Weather_Condition         4
HolidayorPromotion        2
Competitor_Pricing     9751
Seasonality               4
dtype: int64

In [ ]:
X = data.drop(['NObeyesdad'], axis = 1)
y = data['NObeyesdad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
print('Train Size', X_train.shape)
print('Test Size', X_test.shape)


likely the numerical values are Demand_Forecast, Units_Ordered, Units_Sold, Inventory_level, Price, and Competitor_Pricing due to their low cardinality

In [45]:
num_cols = ['Demand_Forecast, Units_Ordered, Units_Sold, Inventory_level, Price, Competitor_Pricing']

listItem= []

for col in num_cols:
  listItem.append([col, round(X_train[col].skew(),1), np.where(
    (round(X_train[col].skew(),1) <= 0.5) & (round(X_train[col].skew(),1) >= -0.5),
    'normal',
    'skewed')])



skewness = pd.DataFrame(columns=['column_name', 'skewness', 'distribution'], data= listItem)
skewness
